In [1]:
# 接收方 2 脚本
from alphafed import mock_context
from alphafed.contractor import ContractEvent, TaskMessageContractor
from alphafed.data_channel import SharedFileDataChannel

# 以下 ID 要与发送方设置相同
task_id = 'b2f615fb-f2e7-4aa0-b5fb-a4fd68c6f38a'
sender_id = 'ca28a832-cc13-40a3-8292-dee4c960a6cb'
receiver_id_1 = '98cf65a2-53dc-4ee4-b261-7cee17f42355'
receiver_id_2 = 'ff433f39-93cf-4bc8-8040-d4392a6fd139'
receiver_ids = [receiver_id_1, receiver_id_2]
contractor = TaskMessageContractor(task_id=task_id)
data_channel = SharedFileDataChannel(contractor=contractor)

def complementary_handler(event: ContractEvent):
    # 处理其它事件
    pass

print('准备开始接收数据')
with mock_context(id=receiver_id_2, nodes=[sender_id, *receiver_ids]):
    print('发送方一对一发送数据')
    source, data_stream = data_channel.receive_stream(receiver=receiver_id_2,
                                                      source=sender_id,
                                                      complementary_handler=complementary_handler)
    print(f'接收到 {source} 发送的消息：`{data_stream.decode()}`')
    print('发送方一对多发送数据')
    # 接收方的接收方式是一样的
    source, data_stream = data_channel.receive_stream(receiver=receiver_id_2,
                                                      source=sender_id,
                                                      complementary_handler=complementary_handler)
    print(f'接收到 {source} 发送的消息：`{data_stream.decode()}`')

    print('发送方与接收方 1 一起向我发送消息。')
    data_dict = data_channel.batch_receive_stream(receiver=receiver_id_2,
                                                  source_list=[sender_id, receiver_id_1],
                                                  complementary_handler=complementary_handler,
                                                  ensure_all_succ=True)
    for _source, _data_stream in data_dict.items():
        print(f'接收到 {_source} 发送的消息：`{_data_stream.decode()}`')

准备开始接收数据
发送方一对一发送数据
接收到 ca28a832-cc13-40a3-8292-dee4c960a6cb 发送的消息：`A message from sender.`
发送方一对多发送数据
接收到 ca28a832-cc13-40a3-8292-dee4c960a6cb 发送的消息：`A message from sender.`
发送方与接收方 1 一起向我发送消息。
接收到 98cf65a2-53dc-4ee4-b261-7cee17f42355 发送的消息：`A message from sender.`
接收到 ca28a832-cc13-40a3-8292-dee4c960a6cb 发送的消息：`A message from sender.`
